# Python Course on Classes and Functional Programming

### *J.A. Hernando, USC, 2016*

## Functional Programming - An example

In [211]:
import time
print ' Last revision ', time.asctime()

 Last revision  Sat Nov  5 16:52:36 2016


### 1. 'El Cinquillo' in FP

We are going to re-write 'El Cinquillo' car game, that we discussed in the Object Oriented notebook, using now Functional Programming (FP).

In FP we have to identify the different functions, and the arguments that are passed between functions.

We can reinterpret an object of type *Card* with a 2-item tuple, one item is the rank, the second the suit. *('5', '0')* is the five of golden coins. A *Desk* will be now simply a list of *(rank, suit)* tuples.
It seems natural that the functions are going to take as argument cards, or list of cards.

Respect the functions. The is a initial function that deal the cards among the players. A function to check is a card can be put on the table. Another to check from the cards on the hand of a player, which ones can be played. An a function that will select one, it can be manually or via a logic. We have almost all... Let's try:


In [159]:
%%writefile cinquillo.py

import random
import copy
import itertools

ranks = ['1', '2', '3', '4', '5', '6', '7', 'sota', 'caballo', 'rey']
suits = ['O', 'C', 'S', 'B']
spanish_desk = [(rank, suit) for rank in ranks for suit in suits]

def order_cards(cards):
    """ order the cards, cards is a list of cards. Each card is a typle (rank, suit).
    Returns the ordered list of cards!
    """
    def order_card(card1, card2):
        s1, s2 = suits.index(card1[1]), suits.index(card2[1])
        if (s1 < s2): 
            return -1  
        elif (s1 > s2):
            return 1
        r1, r2 = ranks.index(card1[0]), ranks.index(card2[0])
        if (r1 < r2):
            return -1
        elif (r1 > r2):
            return 1
        return 0  
    
    return sorted(cards, cmp=order_card)


def deal_cards(nplayers):
    """ Deal the cards among nplayers. 
    Returns nplayer list with cards.
    """
    desk = copy.deepcopy(spanish_desk)
    random.shuffle(desk)
    hands = []
    for i in range(nplayers):
        hands.append([])
    for i in itertools.cycle(range(nplayers)):
        if (len(desk) <= 0):
            break
        hands[i].append(desk.pop())
    return hands


def valid_card(card, table):
    """ check if the card, a tuple (card, suit), can be display on the table. 
    table here is the list of cards on the table (face up).
    Returns true of false.
    """
    O5 = ('5','O')
    if (not O5 in table):
        return (card == O5)
    if (card[0] == '5'):
        return True
    rank, suit = card[0], card[1]
    ipos = ranks.index(rank)
    i0 = max(ipos-1, 0)
    i1 = min(ipos+1, len(ranks)-1)
    iicards = [(ranks[ii], suit) for ii in (i0, i1)]
    return any([(icard in table) for icard in iicards])


def cards_to_play(hand, table):
    """ return a list with the cards than can play from the hand, 
    a list of cards, and the table, the list of cards on the table or display.
    """
    cards = [card for card in hand if valid_card(card, table)]
    return cards
    

def show_cards(cards, name='cards'):
    """ print in the screen the cards that are in cards, a list of cards.
    """
    def str_(card):
        return str(card[0])+'-'+str(card[1])
    print name, str([str_(card) for card in cards])
    return


def give_card(card, pile1, pile2):
    """ transfer the card from pile1 (a list of cards) to pile2 (a second list of cards).
    Returns the new pile1 and pile2 list of cards
    """
    cpile1, cpile2 = copy.deepcopy(pile1), copy.deepcopy(pile2) 
    cpile1.remove(card)
    cpile2.append(card)
    cpile2 = order_cards(cpile2)
    return cpile1, cpile2


def automatic_play(hand, table):
    """ do an automatic play, take a randon card from the hand that can be played.
    hand is the list  of card on the hand of a player, 
    table is the list of cards on table or display.
    returns the new list of cards on the hand and the table
    """
    cards = cards_to_play(hand, table)
    if (len(cards) == 0):
        return hand, table
    card = random.sample(cards, 1)[0]     
    return give_card(card, hand, table)


def manual_play(hand, table):
    """ manually play, ask the user to select a card from the hand, a list of cards, to play.
    table is the list of cards on the table or display.
    Returns the new list of card on the hand of the player and the table.
    """
    show_cards(hand, name='hand:')
    show_cards(table, name='table:')
    cards = cards_to_play(hand, table)
    show_cards(cards, name='select one:')
    while True:
        dat = raw_input('select a card ')
        if ((dat == '') and (len(cards) == 0)):
            return hand, table
        elif (dat.find('-') > 0): 
            rank, suit = (dat.split('-'))
            card = (rank, suit)
            if ((valid_card(card, table)) and (card in hand)):
                return give_card(card, hand, table)
        print 'Not valid card!'
    return


def cinquillo(nplayers, name='Salome'):
    """ plays the 'El cinquillo' game, with nplayers the number of players, 
    and name, the name of the manual player.
    """
    hands, table = deal_cards(nplayers), []
    for iplayer in itertools.cycle(range(nplayers)):
        hand = hands[iplayer]
        issalome = (iplayer%nplayers == 0)
        if (issalome):
            hand, table = manual_play(hand, table)
        else:
            hand, table = automatic_play(hand, table)
        hands[iplayer] = hand
        cname = 'player'+str(iplayer)
        if (issalome): cname = name
        print cname, ' has ', len(hand), 'cards'
        if (len(hand) == 0):
            print cname, ' wins!!'
            break
    return

Writing cinquillo.py


We can test the some of the functions.

In [154]:
# deal cards, show one player
hands = deal_cards(4)
show_cards(hands[0], 'hand0')
show_cards(order_cards(hands[0]), 'hand1')

hand0 ['1-O', '5-O', '2-O', '1-C', 'sota-C', 'caballo-O', '7-C', '4-O', 'sota-O', 'rey-O']
hand1 ['1-O', '2-O', '4-O', '5-O', 'sota-O', 'caballo-O', 'rey-O', '1-C', '7-C', 'sota-C']


In [155]:
# how is the players with the 5-O
for i in range(4):
    show_cards(cards_to_play(hands[i], []), 'player'+str(i))


player0 ['5-O']
player1 []
player2 []
player3 []


In [156]:
# lets do a round, manual
table = []
for iplayer in range(nplayers):
    hands[iplayer], table = manual_play(hands[iplayer], table)

hand: ['1-O', '5-O', '2-O', '1-C', 'sota-C', 'caballo-O', '7-C', '4-O', 'sota-O', 'rey-O']
table: []
select one: ['5-O']
select a card 5-O
hand: ['4-C', '3-B', '3-O', 'sota-B', '3-C', 'caballo-S', '6-S', '5-S', '6-O', '5-C']
table: ['5-O']
select one: ['5-S', '6-O', '5-C']
select a card 5-S
hand: ['6-B', '4-B', '1-B', 'rey-B', '2-C', '2-S', '5-B', '4-S', 'caballo-C', '3-S']
table: ['5-O', '5-S']
select one: ['5-B', '4-S']
select a card 5-B
hand: ['7-B', 'caballo-B', '2-B', 'sota-S', 'rey-C', '7-S', '6-C', '1-S', '7-O', 'rey-S']
table: ['5-O', '5-S', '5-B']
select one: []
select a card 


In [157]:
# lets do another round, automatic
for iplayer in range(nplayers):
    show_cards(table, 'table')
    show_cards(hands[iplayer], 'player'+str(iplayer))
    hands[iplayer], table = automatic_play(hands[iplayer], table)

table ['5-O', '5-S', '5-B']
player0 ['1-O', '2-O', '1-C', 'sota-C', 'caballo-O', '7-C', '4-O', 'sota-O', 'rey-O']
table ['4-O', '5-O', '5-S', '5-B']
player1 ['4-C', '3-B', '3-O', 'sota-B', '3-C', 'caballo-S', '6-S', '6-O', '5-C']
table ['4-O', '5-O', '6-O', '5-S', '5-B']
player2 ['6-B', '4-B', '1-B', 'rey-B', '2-C', '2-S', '4-S', 'caballo-C', '3-S']
table ['4-O', '5-O', '6-O', '5-S', '4-B', '5-B']
player3 ['7-B', 'caballo-B', '2-B', 'sota-S', 'rey-C', '7-S', '6-C', '1-S', '7-O', 'rey-S']


Let's play!

In [158]:
# let's play
cinquillo(4)

hand: ['6-B', '3-C', '5-C', '4-S', '7-C', 'sota-C', '2-O', '2-B', '6-C', '1-O']
table: []
select one: []
select a card 
Salome  has  10 cards
player1  has  10 cards
player2  has  9 cards
player3  has  9 cards
hand: ['6-B', '3-C', '5-C', '4-S', '7-C', 'sota-C', '2-O', '2-B', '6-C', '1-O']
table: ['5-O', '6-O']
select one: ['5-C']
select a card 5-C
Salome  has  9 cards
player1  has  9 cards
player2  has  8 cards
player3  has  9 cards
hand: ['6-B', '3-C', '4-S', '7-C', 'sota-C', '2-O', '2-B', '6-C', '1-O']
table: ['5-O', '6-O', '4-C', '5-C', '5-S']
select one: ['3-C', '4-S', '6-C']
select a card 3-C
Salome  has  8 cards
player1  has  8 cards
player2  has  7 cards
player3  has  9 cards
hand: ['6-B', '4-S', '7-C', 'sota-C', '2-O', '2-B', '6-C', '1-O']
table: ['5-O', '6-O', '2-C', '3-C', '4-C', '5-C', '5-S', '5-B']
select one: ['6-B', '4-S', '6-C']
select a card 6-B
Salome  has  7 cards
player1  has  7 cards
player2  has  6 cards
player3  has  8 cards
hand: ['4-S', '7-C', 'sota-C', '2-O', '2

That's all!